# Freight Rates API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
pip install signal-ocean

In [35]:
signal_ocean_api_key = '' #replace with your subscription key

## Freight Rates API
The Freight Rates API retrieves freight costs breakdown for a given load,
discharge port and vessel class. First create connection towards
Freight Rates API in order to find available ports and vessel classes:

In [36]:
from signal_ocean import Connection, FreightRatesAPI
from signal_ocean.freight_rates import PortFilter

connection = Connection(api_key=signal_ocean_api_key)
fr_api = FreightRatesAPI(connection)

#### Find available ports and vessel classes
Now retrieve the available vessel classes and look if specific ports are available.
If you want to get all the available ports do not pass any parameter to the
corresponding method.

In [38]:
vessel_classes = fr_api.get_vessel_classes()
print(vessel_classes)
cpc = fr_api.get_ports(PortFilter(name_like='CPC'))[0]
augusta = fr_api.get_ports(PortFilter(name_like='Augusta'))[0]

('VLGC', 'MidsizeLGC', 'HandyLPG', 'SmallLPG', 'VLOC', 'Capesize', 'PostPanamaxDry', 'PanamaxDry', 'Supramax', 'Handymax', 'Handysize', 'ULVC', 'NewPanamax', 'PostPanamax', 'Panamax', 'Feedermax', 'Feeder', 'VLCC', 'Suezmax', 'Aframax', 'PanamaxTanker', 'MR2', 'MR1', 'SmallTanker', 'ModernTFTE', 'SmallDry', 'ULCC', 'SmallContainer')


#### Get freight rates for specific ports and vessel class
In this example we retrieve today’s freight rate for Clean Panamax Amsterdam - Lome:

In [39]:
amsterdam = fr_api.get_ports(PortFilter(name_like='Amsterdam'))[0]
lome = fr_api.get_ports(PortFilter(name_like='Lome'))[0]
fr = fr_api.get_freight_pricing(load_port_id=amsterdam.id, discharge_port_id=lome.id,
                                vessel_classes=["PanamaxTanker"], is_clean=True)
print(fr)

(FreightPricing(vessel_class='PanamaxTanker', rate=100.0, rate_type='WorldScale', estimated_flat_rate=14.3205, costs=Cost(canal=0.0, freight_cost=859230.0), total_freight_cost=859230.0, total_freight_rate=14.3205, route_type='Main', load_port=Port(name='Amsterdam', id=None, country='Netherlands', area='Continent'), discharge_port=Port(name='Lome', id=None, country='Togo', area='Africa Atlantic Coast'), quantity=60000.0, min_flat_augusta_used=False, routing_choices=[]),)


We can also plot the freight rates for the same combination from 1st of January
until today:

In [40]:
from datetime import date, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

start_date = date(2021, 1, 1)
end_date = date.today()
delta = end_date - start_date
dates = [start_date + timedelta(days=i) for i in range(delta.days + 1)]

rates = []
for day in dates:
    frates = fr_api.get_freight_pricing(load_port_id=cpc.id,
                                        discharge_port_id=augusta.id,
                                        vessel_classes=["Aframax"],
                                        is_clean=False,
                                        date=day)
    rates.append(frates[0].rate)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.plot(dates, rates)
plt.gcf().autofmt_xdate()